<a href="https://colab.research.google.com/github/ConorD28/NBA-Research/blob/main/NBA_Before_Finals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.decomposition import PCA
%matplotlib inline
df = pd.read_csv('NBA Finals Research_Upload.csv')
#df.drop(df.tail(37).index,
#        inplace = True)
df2 = df.drop(columns=df.columns[0], axis=1)
df_NP = df2.drop(columns=df.columns[-25:], axis=1)

In [ ]:
df2.isnull().sum().sum() #Check if there are NA values

0

In [ ]:
import scipy.stats
def correlation(dataset, threshold, target): #Function to get Pearson's correlation between input and target
  data = []
  for i in range(len(dataset.columns)):
      cor2 = dataset.iloc[:,i].corr(target) #scipy.stats.spearmanr(x, y)[0] and scipy.stats.kendalltau(x, y)[0] or Permutation Importance
      #cor2 = scipy.stats.spearmanr(dataset.iloc[:,i], target[10:30])[0]
      #cor2 = scipy.stats.kendalltau(dataset.iloc[:,i], target[10:30])[0]
      column_headers = list(dataset.columns.values)
      if(abs(cor2) > threshold):
        data.append(dataset.iloc[:,i]) #make list of columns that meet the threshold
        #if(abs(cor2) < .7):
         # print(column_headers[i], f'cor:{cor2:.3f}')
      i = i + 1
  return data

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
from numpy.random.mtrand import random_sample
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, MultiTaskLassoCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
def Scores(y, y_pred, y_full):
  MSE = mean_squared_error(y, y_pred)
  MAE = mean_absolute_error(y, y_pred)
  Normalized_RMSE = (np.sqrt(MSE)/np.mean(y_full))*100
  Normalized_MAE = (MAE/np.mean(y_full))*100
  Avg_Normalized_Score = (Normalized_RMSE + Normalized_MAE)/2
  avg_error = (sum(abs(y_pred-y)))/10
  print(f'Avg. Normalized Score:{ Avg_Normalized_Score:.1f}%')
  print(f'Normalized RMSE:{ Normalized_RMSE:.1f}%')
  print(f'Normalized MAE:{ Normalized_MAE:.2f}%')
  #print(f'MAE:{ MAE:.3f}')
  #print(f'RMSE:{ np.sqrt(MSE):.3f}')
  print(f'Avg. Error:{avg_error:.1f}')
  print(abs(y_pred-y))
  return Avg_Normalized_Score

In [ ]:
def RLE_Model(X, y, choice, predict_df): #Function to run Ridge, Lasso, or ElasticNet model
  #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0) #Train/Test
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]

  if(choice=="Ridge"):
    alphas = np.geomspace(1e-10, 1e10, num=100)
    m1 = RidgeCV(alphas=alphas)
    m1.fit(X_train, y_train)
    print()

  if(choice=="Lasso"):
    alphas = np.geomspace(1e-10, 1e10, num=100)
    m1 = LassoCV(alphas=alphas)
    m1.fit(X_train, y_train)
    print("Lasso alpha: ", m1.alpha_)

  if(choice=="Elastic"):
    l1_ratio = [0, 0.3, 0.5, 0.7, 0.9, 1]
    alphas = np.geomspace(1e-10, 1e10, num=100)
    m1 = ElasticNetCV(alphas=alphas, l1_ratio=l1_ratio, max_iter=100000)
    m1.fit(X_train, y_train)
    print("Elastic alpha: ", m1.alpha_)

  #print(f'Chosen alpha  {pipeline.steps[0][1].alpha_:.6f}')
  #print(f'Intercept (b) {pipeline.steps[0][1].intercept_:.6f}')
  #print(pd.Series(pipeline.steps[0][1].coef_, index=X.columns),'\n')

  #Calculate the predicted values:
  y_train_pred = m1.predict(X_train)
  #print(y_train_pred)
  print()

  y_test_pred = m1.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Test Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  #Predict:
  predictions = m1.predict(predict_df)

  return y_test_pred, y_train_pred, predictions

In [ ]:
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

def GBR_model(X,y, t, l, n, predict_df):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  reg = GradientBoostingRegressor(tol = t, learning_rate = l, n_estimators=n, random_state=0) #default: tol = 0.0001, learning rate - 0.1, 100, friedman_mse
  reg.fit(X_train, y_train)
  y_train_pred = reg.predict(X_train)
  #print(y_train_pred)

  y_test_pred = reg.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  #Predict:
  predictions = reg.predict(predict_df)
  return y_test_pred, y_train_pred, predictions

In [ ]:
from sklearn.ensemble import BaggingRegressor
import xgboost as xgb
import pickle

def BR_model(X,y):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  reg = BaggingRegressor(base_estimator=xgb.XGBRegressor())
  #reg = pickle.load(open("BR_model2", "rb"))
  reg.fit(X_train, y_train)
  y_train_pred = reg.predict(X_train)
  #print(y_train_pred)

  y_test_pred = reg.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  #Predict:
  #predictions = reg.predict(predict_df)
  #print(predictions)

  pickle.dump(reg, open("BR_model", "wb"))
  return y_test_pred, y_train_pred

In [ ]:
from sklearn.linear_model import SGDRegressor
#SGD Regressor:
def SGD_model(X,y, t, ep, predict_df):

  reg = SGDRegressor(max_iter=1000, tol=t, epsilon = ep) #tol = 0.001, epsilon=0.1
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  reg.fit(X_train, y_train)
  y_train_pred = reg.predict(X_train)
  #print(y_train_pred)

  y_test_pred = reg.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  predictions = reg.predict(predict_df)
  #print(predictions)

  pickle.dump(reg, open("SGD_model", "wb"))
  return y_test_pred, y_train_pred, predictions

In [ ]:
#Keras Sequential Neural Net
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=2)

def Keras_model(X,y,e, u, u2, u3, u4, u5, choice):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  model = Sequential()
  model.add(Dense(u, input_dim=X_train.shape[1], activation='relu')) # Hidden 1, 60
  model.add(Dense(units=u2,activation='relu')) # Hidden 2, 30
  model.add(Dense(units=u3,activation='relu'))
  model.add(Dense(units=u4,activation='relu'))
  model.add(Dense(units=u5,activation='relu'))
  model.add(Dense(units=15,activation='relu')) #15
  model.add(Dense(units=1)) #,activation='relu'
  model.compile(loss='mean_squared_error', optimizer=choice) #
  model.fit(X_train, y_train, verbose=0, epochs=e, callbacks=[early_stop]); #callbacks=[early_stop]

  y_train_pred = model.predict(X_train)
  y_train_pred = y_train_pred.flatten()

  y_test_pred = model.predict(X_test)
  y_test_pred = y_test_pred.flatten()

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  #print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  model.save('/content/drive/MyDrive/Models/Keras_Model', save_format="h5")

In [ ]:
from sklearn.tree import DecisionTreeRegressor

def DTR_model(X,y,leafs):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  # We introduce regularization by increasing the value of min_samples_leaf
  tree_reg_regularized = DecisionTreeRegressor(random_state=42, min_samples_leaf=leafs)
  tree_reg_regularized.fit(X_train, y_train)
  y_train_pred = tree_reg_regularized.predict(X_train) #_regularized
  #print(y_train_pred)

  y_test_pred = tree_reg_regularized.predict(X_test) #_regularized

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

In [ ]:
from sklearn.svm import LinearSVR

def SVM_model(X,y,ep):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  svm_reg = LinearSVR(epsilon=ep, random_state=42) #default: epsilon = 0 tol=0.0001, C=1.0
  svm_reg.fit(X_train, y_train)

  #Train Predictions:
  y_train_pred = svm_reg.predict(X_train)
  #print(y_train_pred)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Test Predictions:
  y_test_pred = svm_reg.predict(X_test)
  print("Test predictions:")
  #print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

In [ ]:
from sklearn.svm import SVR

def SVM_models(X,y, choice, ep, C_value, predict_df):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  if(choice=="rbf"):
    model = SVR(kernel="rbf", C=C_value, gamma=0.1, epsilon=ep) #0.1 default ep; 100 default C, 0.1 default gamma
    model.fit(X_train, y_train)

  if(choice=="poly"):
    model = SVR(kernel="poly", C=C_value, gamma="auto", degree=3, epsilon=ep, coef0=1) #0.1 default ep; 100 default C
    model.fit(X_train, y_train)

  if(choice == "linear"):
    model = SVR(kernel="linear", C=C_value, gamma="auto", degree=3, epsilon=ep, coef0=1) #0.1 default ep; 100 default C
    model.fit(X_train, y_train)

  #Train Predictions:
  y_train_pred = model.predict(X_train)
  #print(y_train_pred)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Test Predictions:
  y_test_pred = model.predict(X_test)
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.2f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  final_preds = model.predict(predict_df)

  return y_test_pred, y_train_pred, final_preds

In [ ]:
from sklearn.datasets import make_swiss_roll
from sklearn.manifold import LocallyLinearEmbedding

In [ ]:
#df.iloc[:, 1110:]

In [ ]:
#Feature Importance:
scaler = StandardScaler() #MinMaxScaler, StandardScaler
X_train, X_test, y_train = df_NP[10:30], df_NP[0:10], df[10:30]
data_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

data_scaled.drop(["SOS", "SOS_Use", "EFGPercPerSOS_Per 100_Clutch_Plr", "FGPercPerSOS_Per 100_Clutch_Plr",
                  "FGPerc_5_9 FT._Last10.1", "PACE_Per 100_Plr", "ROAD SOS", "FGPercPerSOS_10_14 FT._Last10",
                  "FGAPerSOS_5_9 FT._Last10", 'FGPercPerSOS_Above the Break 3', 'PACE_Per 100_vsOwnConf_Plr',
                  'FGPerc_Right Corner 3', 'FGMPerSOS_Mid Range', 'FGPercPerSOS_20_24 FT._vsOwnC',
                  'WSPer48PerSOS_Per 100_Plr', 'EFGPercPerSOS_Per 100_vsOwnConf_Plr',
                  'TD3_Per 100_Clutch_Plr', 'FGAPerSOS_10_14 FT.', 'FGPercPerSOS_10_14 FT.',
       'FGPercPerSOS_20_24 FT.'],axis=1,inplace = True) #dropped stats for EST DEF RTG

data_correlated = correlation(data_scaled, 0.561, y_train['EST_DEFRTG_Pfs']) #
data_correlated_df = pd.DataFrame(data_correlated)
data_correlated_df2 = data_correlated_df.transpose() #Correlated inputs

#Variables Added to EST Def Rtg:
data_correlated_df2.insert(21,"EST. DEFRTG",data_scaled["EST. DEFRTG"])
data_correlated_df2.insert(22,"OPP EFGPercPerSOS_PerGM",data_scaled["OPP EFGPerc_PerGM"])
data_correlated_df2.insert(23,"OPP TOVPerc_PerGM",data_scaled["OPP TOVPerc_PerGM"])
data_correlated_df2.insert(24,"OPP FBPSPerSOS_Per 100",data_scaled["OPP FBPSPerSOS_Per 100"])
data_correlated_df2.insert(25,"OPP PITPPerSOS_Per 48_Clutch",data_scaled["OPP PITPPerSOS_Per 48_Clutch"])
data_correlated_df2.insert(26,"OPP PITP PerSOS_Per 100",data_scaled["OPP PITP PerSOS_Per 100"])
data_correlated_df2.insert(27,"PP100PerSOS_LA",data_scaled["PP100PerSOS_LA"])

X = df_NP.loc[:, data_correlated_df2.columns] #get non scaled data with important features

#Train test split and scale:
X_train, X_test = X[10:30], X[0:10] #split non-scaled important variables
X_train_processed = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test_processed = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
correlated_scaled_data = pd.merge(X_test_processed, X_train_processed, how = 'outer')
#correlated_scaled_data_MMS = pd.merge(X_test_processed, X_train_processed, how = 'outer')

#PCA:
pca=PCA(n_components = 2)
X_train_processed_PCA = pca.fit_transform(X_train_processed)
X_train_PCA_df = pd.DataFrame(X_train_processed_PCA)
X_test_processed_PCA = pca.transform(X_test_processed)
X_test_PCA_df = pd.DataFrame(X_test_processed_PCA)
data_PCA = pd.merge(X_test_PCA_df, X_train_PCA_df, how = 'outer')
#data_PCA_MMS = pd.merge(X_test_PCA_df, X_train_PCA_df, how = 'outer')

print("Principal axes:\n", pca.components_.tolist())
print("Explained variance:\n", pca.explained_variance_.tolist())
print("Mean:", pca.mean_)

#LLE:
#X_swiss, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=19, random_state=42) #n_components=2 is default, neighbors 5 is default
X_unrolled_train = lle.fit_transform(X_train_processed)
X_train_LLE_df = pd.DataFrame(X_unrolled_train)
X_unrolled_test = lle.transform(X_test_processed)
X_test_LLE_df = pd.DataFrame(X_unrolled_test)
data_LLE = pd.merge(X_test_LLE_df, X_train_LLE_df, how = 'outer')
#data_LLE_19 = pd.merge(X_test_LLE_df, X_train_LLE_df, how = 'outer')
#data_LLE_MMS = pd.merge(X_test_LLE_df, X_train_LLE_df, how = 'outer')
#data_LLE_MMS_19 = pd.merge(X_test_LLE_df, X_train_LLE_df, how = 'outer')

X = correlated_scaled_data #try correlated_scaled_data, data_PCA, data_LLE
y = df2['EST_DEFRTG_Pfs'] #

In [ ]:
#dropped stats for EST OFF RTG
data_scaled.drop(["NETRTGPerSOS_Per 100_Clutch_Plr", "DEFRTG_Per 100_Clutch_Plr",
                  "NETRTG_Per 100_Clutch_Plr", "DREBPerSOS_Per 100_Clutch_Plr",
                  "DREB_Per 100_Clutch_Plr", '+Per_PerSOS_Per 100_Clutch_Plr',
                  'DD2PerSOS_Per 100_Clutch_Plr', "+Per__Per 100_Clutch_Plr",
                  'OPP PTSPerSOS_Per 100_Clutch', 'oFGA_Above the Break 3', 'Seed',
                  'DREBPerSOS_Per 100_Clutch_Plr', 'oFGA_20_24 FT._Last10'],axis=1,inplace = True)

#Variables Added to EST Off Rtg:
data_correlated_df2.insert(26,"PTS_Per 48",data_scaled["PTS_Per 48"])
data_correlated_df2.insert(27,"EST. OFFRTGPerSOS",data_scaled["EST. OFFRTGPerSOS"])
data_correlated_df2.insert(28,"FGMPerSOS_Corner 3",data_scaled["FGMPerSOS_Corner 3"])
data_correlated_df2.insert(29,"PP100PerSOS_LA",data_scaled["PP100PerSOS_LA"]) #
data_correlated_df2.insert(30,"eFG+_Plr",data_scaled["eFG+_Plr"]) #
data_correlated_df2.insert(31,"FGPerc_10_14 FT._Last10.1", data_scaled["FGPerc_10_14 FT._Last10.1"])
data_correlated_df2.insert(32,"AST RATIO_Per 100_vsOwnConf_Plr", data_scaled["AST RATIO_Per 100_vsOwnConf_Plr"])
data_correlated_df2.insert(33,"FTAPerSOS_Per 100_Clutch", data_scaled["FTAPerSOS_Per 100_Clutch"])
data_correlated_df2.insert(34,"TSPerc_Per 48_Clutch", data_scaled["TSPerc_Per 48_Clutch"])
data_correlated_df2.insert(35,"FGA_<5 FT.",data_scaled["FGA_<5 FT."])
data_correlated_df2.insert(36,"AST_Per 100_vsOwnConf_Plr",data_scaled["AST_Per 100_vsOwnConf_Plr"]) #
data_correlated_df2.insert(37,"FGA_RA",data_scaled["FGA_RA"]) #eFG+_Plr
data_correlated_df2.insert(38,"ASTPerTO_Per 48_Clutch",data_scaled["ASTPerTO_Per 48_Clutch"]) #
data_correlated_df2.insert(39,"AST Ratio_Per 100 Plays_vsOwnC",data_scaled["AST Ratio_Per 100 Plays_vsOwnC"]) #

In [ ]:
#Model: .6
RLE_Model(X, y, "Ridge", X) #
RLE_Model(X, y, "Lasso", X) #
SGD_model(X,y, 1e-3, 0.1, X) #
GBR_model(X,y, .0001, 0.01, 100, X) #
print("5:")
DTR_model(X,y, 100) #default is 100

In [ ]:
SVM_models(X, y, "rbf", 3, 100, X) #3.7%, 1.9%, corr_MMS 2.3
SVM_models(X, y, "poly", 2.2, 100, X) #3.9%, 1.1%, LLE_19 4.5, 3.9%, 1.5%, PCA_MMS 1.9, 3%, 1.3%, corr_MMS 2.1
SVM_models(X, y, "linear", 2.9, 100, X) #3.2%, .9%, corr, 3.6%, .5%, PCA 5.1, 3.8%, .6%, PCA_MMS 5.1, 3.2%, .9%, corr_MMS
SVM_model(X,y, 0) #

In [ ]:
SVM_models(X, y, "poly", 3.2, 100, X)
SVM_models(X, y, "linear", 1.9, 100, X)
SVM_models(X, y, "poly", 4.2, 100, X)
SVM_models(X, y, "linear", 3.9, 100, X)

In [ ]:
RLE_Model(X, y, "Elastic", X) #
BR_model(X,y) #
Keras_model(X, y, 200, 120, 60, 30, 20, 15, "adamax") #
#adam, nadam; adamax

**Estimated** **Ratings**

In [ ]:
#EST_OFFRTG_Pfs Blender: #3.1%, 1%, 3.0% avg error, .6
est_offrtg = pd.read_csv('EST_OFFRTG Inputs.csv')
scaler = StandardScaler()
train_processed = scaler.fit_transform(X_train)
scaled_data = scaler.transform(est_offrtg)
scaled_data_df = pd.DataFrame(scaled_data)

scaler_MMS = MinMaxScaler()
train_processed_MMS = scaler_MMS.fit_transform(X_train)
scaled_data_MMS = scaler_MMS.transform(est_offrtg)
scaled_data_MMS_df = pd.DataFrame(scaled_data_MMS)

X = correlated_scaled_data
X_corr_MMS = correlated_scaled_data_MMS

preds = SVM_models(X_corr_MMS, y, "poly", 2.1, 100, scaled_data_MMS_df) #3%, 1.3%, corr_MMS
preds2 = SVM_models(X, y, "linear", 2.9, 100, scaled_data_df) #3.2%, .9%, corr
preds3 = SVM_models(X_corr_MMS, y, "linear", 2.9, 100, scaled_data_MMS_df) #3.2%, .9%, corr_MMS

#Get stats on blender
train_preds = (preds[1] + preds2[1] + preds3[1])/3

test_preds = (preds[0] + preds2[0] + preds3[0])/3

print("Blender Train Scores then Test Scores:")
Scores(y[10:30], train_preds, y)
print()
Scores(y[0:10], test_preds, y)
print()

est_off_rtg_predictions = pd.DataFrame()
est_off_rtg_predictions["Predictions"] = (preds[2] + preds2[2] + preds3[2])/3
#est_off_rtg_predictions.to_excel("EST_OFF_RTG_Predictions.xlsx")

In [ ]:
est_off_rtg_predictions["Predictions"]

In [ ]:
#EST_DEFRTG_Pfs Blender: #2.3%, .8%, 2.2% avg error, .561
est_def_rtg = pd.read_csv('EST_DEFRTG Inputs.csv')
scaler = StandardScaler()
train_processed = scaler.fit_transform(X_train)
scaled_data = scaler.transform(est_def_rtg)
scaled_data_df = pd.DataFrame(scaled_data)

lle.fit_transform(X_train)
scaled_data_LLE = lle.transform(scaled_data)
scaled_data_LLE_df = pd.DataFrame(scaled_data_LLE)

scaler_MMS = MinMaxScaler()
scaler_MMS.fit_transform(X_train)
scaled_data_MMS = scaler_MMS.transform(est_def_rtg)
scaled_data_MMS_df = pd.DataFrame(scaled_data_MMS)

X = correlated_scaled_data
#X_PCA = data_PCA
X_corr_MMS = correlated_scaled_data_MMS
#X_PCA_MMS = data_PCA_MMS
X_LLE_19 = data_LLE_19
#X_LLE_19_MMS = data_LLE_MMS_19

loaded_BR_model = pickle.load(open("BR_model_LLE_DEF", "rb")) #

preds = (loaded_BR_model.predict(X_LLE_19[0:10]), loaded_BR_model.predict(X_LLE_19[10:30]), loaded_BR_model.predict(scaled_data_LLE)) #1.7%, .8% (21), LLE_19
preds2 = SVM_models(X, y, "linear", 3.5, 100, scaled_data_df) #3.1%, .8% , corr
#preds3 = SVM_models(X_corr_MMS, y, "linear", 3.5, 100, scaled_data_MMS_df) #3.2%, .9%, corr_MMS
#preds4 = SVM_models(X_PCA_MMS, y, "linear", 4.7, 100, X_PCA_MMS) #3.2%, .6% (35), PCA_MMS

#Get stats on blender
train_preds = (preds[1] + preds2[1])/2

test_preds = (preds[0] + preds2[0])/2

print("Blender Train Scores then Test Scores:")
Scores(y[10:30], train_preds, y)
print()
Scores(y[0:10], test_preds, y)
print()

est_def_rtg_predictions = pd.DataFrame()
est_def_rtg_predictions["Predictions"] = (preds[2] + preds2[2])/2
#est_def_rtg_predictions.to_excel("EST_DEF_RTG_Predictions.xlsx")

In [ ]:
est_def_rtg_predictions["Predictions"]

0    98.156553
1    98.668559
Name: Predictions, dtype: float64